In [1]:
import numpy as np
import data

In [2]:
train = data.train
test = data.test

In [3]:
class MatrixFactorization():
    
    def __init__(self, train, test, k, learning_rate, epochs, verbose = False):
        """
        param R : Rating Matrix
        param k : latent parameter
        param learning_rate : alpha on weight update
        param epochs : training epochs
        param verbose : print status
        """
        
        self._R = train
        self._test = test
        self._I = np.array(np.vectorize(lambda x: 0 if x==0 else 1)(train), dtype = np.float64) # indicator matrix
        self._n_user_rated = np.sum(self._I, axis = 1)
        self._n_item_rated = np.sum(self._I, axis = 0)
        self._num_users, self._num_items = train.shape
        # sigma?
        self._lambda = 0.01
        self._k = k
        self._learning_rate = learning_rate
        self._epochs = epochs
        self._verbose = verbose
        
        
    def fit(self):
        """
        training Matrix Factorization : update matrix latent weight and bias
        """
        # init latent features
        self._Y = np.random.normal(0, 0.1, size=(self._num_users, self._k))
        self._V = np.random.normal(0, 0.1, size=(self._num_items, self._k))
        self._W = np.random.normal(0, 0.1, size=(self._num_items, self._k))
        
        self._training_process = []
        for epoch in range(self._epochs):
            for i in range(self._num_users):
                for j in range(self._num_items):
                    if self._R[i, j] > 0 :
                        self.gradient_descent(i, j, self._R[i, j])
                        
            train_cost, test_cost = self.cost()
            self._training_process.append((epoch, train_cost, test_cost))
            
            rank = self.compute_rank()
            print("Iteration : %d, train_cost = %.4f, test_cost = %.4f, rank = %.4f" % (epoch+1, train_cost, test_cost, rank))
                
    
    def cost(self):
        """
        compute RMSE
        """
        xi, yi = self._R.nonzero() # 0 이 아닌 값의 index 반환
        test_x, test_y = self._test.nonzero()
        predicted = self.get_complete_matrix()
        cost_train = 0
        cost_test = 0
        
        for x, y in zip(xi, yi):
            cost_train += pow(self._R[x, y] - predicted[x, y], 2)
        
        for x, y in zip(test_x, test_y):
            cost_test += pow(self._test[x, y] - predicted[x, y], 2)
        
        return np.sqrt(cost_train/len(xi)), np.sqrt(cost_test/len(test_x))
    
    
    def gradient_descent(self, i, j, rating):
        """
        gradient descent function
        param i : user index
        param j : item index
        param rating : rating of (i, j)
        """
        prediction = self.get_prediction(i, j)
        error = rating - prediction
        
        # self._U[i, :] += self._learning_rate * ( error * self._V[j, :] - self._lambda_U / self._n_user_rated[i] * self._U[i, :])
        # self._V[j, :] += self._learning_rate * ( error * self._U[i, :] - self._lambda_V / self._n_item_rated[j] * self._V[j, :])
        
        self._Y[i, :] += self._learning_rate * (error * self._V[j, :] - self._lambda * self._Y[i, :])
        self._V[j, :] += self._learning_rate * (error * (self._Y[i, :] + self._I[i, :].dot(self._W)/self._n_user_rated[i]) - self._lambda * self._V[j, :])
        self._W += self._learning_rate * (error * np.outer(self._I[i, :], self._V[j, :])/self._n_user_rated[i])
        self._W[j, :] -= self._learning_rate * self._lambda * self._W[j, :]
        
    
    def get_prediction(self, i, j):
        """
        get predicted rating by user i on item j
        param i : user index
        param j : item index
        """
        
        return np.dot((self._Y[i, :] + self._I[i, :].dot(self._W)/self._n_user_rated[i]), self._V[j, :])
    
    
    def get_complete_matrix(self):
        """
        compute complete matrix
        """
        
        return np.dot((self._Y + self._I.dot(self._W) / self._n_user_rated[:, np.newaxis]), self._V.T)
    
    
    def compute_rank(self):
        
        prediction = self.get_complete_matrix()
        test_x = np.unique(self._test.nonzero()[0])
        temp_1 = 0
        temp_2 = 0
        
        for x in test_x :
            temp_y = self._test[x].nonzero()
            inv_pre = -1 * prediction[x, temp_y]
            sort_x = inv_pre.argsort() # index starts with 0
            sort_x = sort_x.argsort()
            rank_x = sort_x / len(sort_x[0])
            
            temp_1 += (self._test[x, temp_y] * rank_x).sum()
            temp_2 += self._test[x, temp_y].sum()
        
        rank = temp_1 / temp_2
            
        return rank
    
    
    def print_results(self):
        """
        print fit results
        """

        print("Final R matrix:")
        print(self.get_complete_matrix())
        print("Final RMSE:")
        print(self._training_process[self._epochs-1][2])

In [4]:
np.random.seed(7)
    
np.seterr(all="warn")
    
factorizer = MatrixFactorization(train, test, k=40, learning_rate=0.001, epochs=100, verbose=True)
# regression parameter 2개
factorizer.fit()
factorizer.print_results()

Iteration : 1, train_cost = 3.6970, test_cost = 3.7199, rank = 0.4885
Iteration : 2, train_cost = 3.6870, test_cost = 3.7146, rank = 0.4874
Iteration : 3, train_cost = 3.6522, test_cost = 3.6911, rank = 0.4828
Iteration : 4, train_cost = 3.5204, test_cost = 3.5944, rank = 0.4725
Iteration : 5, train_cost = 3.1124, test_cost = 3.2750, rank = 0.4665
Iteration : 6, train_cost = 2.4254, test_cost = 2.6604, rank = 0.4650
Iteration : 7, train_cost = 1.8994, test_cost = 2.0955, rank = 0.4639
Iteration : 8, train_cost = 1.6035, test_cost = 1.7526, rank = 0.4624
Iteration : 9, train_cost = 1.4239, test_cost = 1.5473, rank = 0.4607
Iteration : 10, train_cost = 1.3066, test_cost = 1.4151, rank = 0.4592
Iteration : 11, train_cost = 1.2258, test_cost = 1.3246, rank = 0.4580
Iteration : 12, train_cost = 1.1677, test_cost = 1.2598, rank = 0.4571
Iteration : 13, train_cost = 1.1244, test_cost = 1.2117, rank = 0.4562
Iteration : 14, train_cost = 1.0911, test_cost = 1.1749, rank = 0.4556
Iteration : 15,